In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus']=False
%matplotlib inline

# 1、载入数据，获取数据

In [4]:
df=pd.read_csv('changsha_head.csv',encoding='utf-8')

In [6]:
df.head()

职位名称                 公司名称 工作城市     薪资范围 学历要求 公司类型  \
0             招商经理/主管    华谊兄弟(长沙)电影文化城有限公司   长沙    5千-8千   本科   其它   
1           商务BD(MCN)            快乐购有限责任公司   长沙  1万-1.5万   本科   其它   
2              区域销售推广                 万达信息   长沙  8千-1.5万   本科   其它   
3             农产品销售经理  红星实业集团有限公司红星农副产品大市场   长沙    8千-1万   本科   其它   
4  5G消息运营商BD(前期需驻场移动)                 小源科技   长沙  1万-1.5万   本科   其它   

         公司规模  职位类别  工作经验                                      福利待遇  
0    100-299人  销售业务  3-5年                      五险一金,包住,包吃,免费班车,免费停车  
1    500-999人  销售业务  3-5年   五险一金,绩效奖金,带薪年假,定期体检,节日福利,年底双薪,补充医疗保险,餐补  
2  1000-9999人  销售管理  3-5年                       五险一金,绩效奖金,带薪年假,定期体检  
3    100-299人  销售业务  3-5年        五险一金,年底双薪,带薪年假,定期体检,高温补贴,节日福利,绩效奖金  
4    100-299人  销售业务  3-5年  五险一金,员工旅游,餐补,交通补助,公司重点项目,节日福利,通讯补助,试用期全额

预览数据

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13479 entries, 0 to 13478
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   职位名称    13479 non-null  object
 1   公司名称    13479 non-null  object
 2   工作城市    13479 non-null  object
 3   薪资范围    13479 non-null  object
 4   学历要求    13479 non-null  object
 5   公司类型    13479 non-null  object
 6   公司规模    13478 non-null  object
 7   职位类别    13479 non-null  object
 8   工作经验    13479 non-null  object
 9   福利待遇    13048 non-null  object
dtypes: object(10)
memory usage: 1.0+ MB


可以看到，该表格有XXX行，XXX列，其中XXX、XXX、XXX、.....列存在缺失值。

拿到一个表格，首先得熟悉行和列所表示的意思，先看懂表格。这里简单注释下各列的中文意思:

XXX(En):XXX(Ch)

# 2、初步的数据清洗与加工

本次分析的数据为（职位）招聘数据，初步分析可得XXXId列的值应为唯一值，那么可通过它来过滤重复发布的（职位）。unique()配合len()可得唯一值（非重复职位）的数量。

In [ ]:
len(df.XXXId.unique())

得到有XXX个唯一的职位，其他多出来的都是重复职位，这就需要去重了。一般会用drop_duplicates()方法去重。

In [ ]:
df_duplicated = df.drop_duplicates(['XXXId'])  #删除列XXXId的重复值

In [ ]:
df_duplicated

In [ ]:
df_duplicated.info()

看看是否有此情况：

一般而言XXXsalary列的值比较混乱，有数字有字符串，我们需要加工薪水数据，把薪水的上限下限隔离出来，以便于统计。观察可得，大部分的薪水是含有上下限的，单位为小写的k，中间用“-”作为连接的字符串，个别职位会出现“k以上”的描述，或者大写的K。这里用apply方法传入自定义的函数进行数据加工。

In [ ]:
#返回薪资上、下跟值,载认返回最低薪资值
def get_Salary(str, method='bottom'):
    #包含 “一” 符号,则返回其索引位置，否则返回一1
    pos = str.find('-')          
    if pos != -1:
        #薪资下值
        botSalary = str[:pos-1]
        #薪资上版值
        topSalary = str[pos+1:len(str)-1]       
    else:
        #针对 “K以上” 形式的值
        botSalary = str[:str.upper().find('K')]
        topSalary = botSalary

    if method == 'bottom':
        #传入除了 “bottom” 以外的任意字符串，则返回topSalary
        return botSalary
    else:
        return topSalary


In [ ]:
df_duplicated['bottomSalary'] = df_duplicated.salary.apply(get_Salary).astype('int')
df_duplicated['topSalary'] = df_duplicated.salary.apply(get_Salary, method= 'top').astype('int')
df_duplicated

有了bottomSalary和topSalary两列，则可以求取薪资的平均值。

In [ ]:
df_duplicated['avgSalary'] = df_duplicated.apply(lambda x: (x.bottomSalary + x.topSalary)/ 2, axis=1)
df_duplicated

现在筛选出众多列中需要分析的部分，以简化后续分析难度。

In [ ]:
df_endData = df_duplicated[['city','companyShortlane','compaySize','education','positionMane','positiouLlables','workTear','argSalary']]
df_endData.head()

# 3、开始分析

### 3、1 总体描述统计
先尝试对一些列进行描述统计，大概了解下数据分布情况。

In [ ]:
df_endData.XXX1.value.counts()

In [ ]:
df_endData.XXX2.value.counts()

In [ ]:
df_endData.XXX3.value.counts()

另外，可以对平均薪资(或有数据的字段)做一个汇总统计，方便查看（薪资）分布情况。

In [ ]:
df_endData.avgSalary.describe()

可以看出，全国XXX岗位的平均薪资在XXXk左右，中位数在XXXk左右，最高薪资是XXXk，并且标准差为XXX，波动较大，部分高新职位拉高了平均值。

### 3、2 可视化

这里将各个岗位(平均薪资)的情况用柱状图可视化。

In [ ]:
df_endData.avgSalary.hist(bins=20)  #平均薪资柱状图

可以看出，(平均薪资)在XXXk上下和XXXk上下的岗位居多，那么是什么因素影响了这一分布呢，我们可以尝试结合不同学历、不同经验的人群进行分析

In [ ]:
#箱线图:不同城市的乎均薪资分布情况(字段自行更改)
df_endData.boxplot(column='avgSalary' , by='city' ,figsize=(13,8))

In [ ]:
#箱线图:不同学历的乎均薪资分布嫣况（字段自行更改）
df_endData.boxplot(column='avgSalary' , by='education',figsize=(13,8))

In [ ]:
#箱线图:不同工作经验的乎均薪资分布情况（字段自行更改）
df_endData.boxplot(column='avgSalary' , by='workYear',figsize=(13,8))

再利用groupby方法进行分组进行多维度分析

In [ ]:
pd.DataFrame(df_endData.groupby('city').avgsalary.mean())

In [ ]:
df_endData. groupby('city').avgSalary.mean().plot.bar(figsize=(13,8))

不同城市、不同学历背景下的平均薪资

In [ ]:
df_endData.groupby(['city','education']).mean().unstack().plot.bar(figsize=(13,8))

In [8]:
##### 为简化问题，如果是分析某个(单一)岗位的岗位性价比以及价值，我们可以以job列的值应为唯一值，那么可通过它来过滤重复发布的职位

In [ ]:
len(df.job.unique())

In [ ]:
data = df.drop_duplicates(['job'])  #删除列job的重复值
data

In [ ]:
data.info()

In [ ]:
df['avg_amt'] = (df['min_amt']+df['max_amt'])/2

In [ ]:
#数据类型转换
df['max_amt'] = df['max_amt'].astype('float')
df.loc[data['salary'].str.contains('万'),'max_amt'] = df['max_amt']*10000
df.loc[data['salary'].str.contains('千'),'max_amt'] = df['max_amt']*1000

df['min_amt'] = data['min_amt'].astype('float')
df.loc[df['salary'].str.contains('万'),'min_amt'] = df['min_amt']*10000
df.loc[df['salary'].str.contains('千'),'min_amt'] = df['min_amt']*1000

In [ ]:
#最小薪资
df['min_amt']=df['salary'].str.split('-').str[0]
#data['min_amt'].astype('float')
#最大薪资
df['max_amt'] = df['salary'].str.split('-').str[1].str.slice(0,4).str.split('万').str[0].str.split('千').str[0]
#将缺失值赋值
#df.loc[df.amtdate.isnull() , 'amtdate' ] = '面议'
df.loc[df.min_amt=='' , 'min_amt' ] = '0'
df.loc[df.max_amt.isnull() , 'max_amt' ] = '0'
df.head()

In [9]:
def get_Salary(str, method='bottom'):
    #包含 “一” 符号,则返回其索引位置，否则返回一1
    pos = str.find('-')          
    if pos != -1:
        #薪资下值
            botSalary = str[:pos-1]
        #薪资上值
            topSalary = str[pos+1:len(str)-1]     
    else:
        #针对面议
        botSalary = 0
        topSalary = 0

    if method == 'bottom':
        #传入除了 “bottom” 以外的任意字符串，则返回topSalary
        return botSalary
    else:
        return topSalary

def isWang(str):
    pos = str.find('-')          
    if pos != -1:
        if str[pos-1]=='万' and str[-1]=='万':
            return 0
        elif str[pos-1]=='千' and str[-1]=='万':
            return 1
        else :
            return 2

In [10]:
if(df.salary.apply(isWang)==0):
    df['bottomSalary'] = df.salary.apply(get_Salary).astype('float')*10
    df['topSalary'] = df.salary.apply(get_Salary, method= 'top').astype('float')*10
elif(df.salary.apply(isWang)==1):
    df['bottomSalary'] = df.salary.apply(get_Salary).astype('float')
    df['topSalary'] = df.salary.apply(get_Salary, method= 'top').astype('float')*10
else:
    df['bottomSalary'] = df.salary.apply(get_Salary).astype('float')
    df['topSalary'] = df.salary.apply(get_Salary, method= 'top').astype('float')

AttributeError: 'DataFrame' object has no attribute 'salary'

In [ ]:
first_s = df.job #sr方法进行字符串邀取
second_s = first_s.str.replace(' ','')  #replace方法剧除空格
third_s = second_s.dropna().str.split(',').apply(pd. value_counts) #以逗号作为分隔符，统计某个关键词在备个职位出现的次数
fourth_s = third_s.unstack().dropna().reset_index() #行列转换后后剧除NaN值,并重置索引
fourth_s

In [ ]:
from wordcloud import WordCloud
fourth_s.index = fourth_s.level_0.str.replace("'",'') #删除单引号
wordcloud = Wordcloud(width=-900 , height=400, background_color='white') #初始化
f, axes = plt.subplots(figsize=(15,10))
#绘图区域
wordcloud.fit_words(fourth_s.level_1)
#祖据词频生版词云
axes = plt.imshow (wordcloud)
#展示词云
plt.axis('off')
#不显示x、y轴
plt.show ()